In [2]:
from openai import OpenAI
import os
from PyPDF2 import PdfReader
import glob
from tqdm import tqdm

os.environ["OPENAI_API_KEY"] = # ENTER_HERE

client = OpenAI()

system_prompt = \
"""## Roal
You are a helpful finical report assistant designed to read annual finatial report and output question answewr pairs in JSON.

## Output JSON format
{
  "question_1": "Answer1",
  "question_2": "Answer2",
  "question_3": "Answer3", 
  ...
}
"""
user_prompt = """
Based on the annual finatial report, generate 3 questions with specific answers that you pretty sure are correct. The answer is better to be a number or phrase that can be easily verified.
The content of the annual finatial report is in below:
"""


for pdf in tqdm(glob.glob("./docs/*.pdf")):
    pdf_reader = PdfReader(pdf)
    text = ""
    for i, page in enumerate(pdf_reader.pages):
        if i < 5:
            text += page.extract_text()
    
    response = client.chat.completions.create(
                model="gpt-3.5-turbo-16k",
                response_format={ "type": "json_object" },
                messages=[
                  {"role": "system", "content": system_prompt},
                  {"role": "user", "content":  user_prompt + text}
                ]
              )
    print(response.choices[0].message.content)
    break

  0%|          | 0/15 [00:05<?, ?it/s]


BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 16385 tokens. However, your messages resulted in 72449 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}

In [10]:
# Upload the user provided file to OpenAI
message_file = client.files.create(
  file=open(r"docs\McDonald 2023 Annual Report_vf.pdf", "rb"), purpose="assistants"
)
 
# Create a thread and attach the file to the message
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "Generate 3 questions with specific answers.",
      # Attach the new file to the message.
      "attachments": [
        { "file_id": message_file.id, "tools": [{"type": "file_search"}] }
      ],
    }
  ]
)
 
# The thread now has a vector store with that file in its tool resources.
print(thread.tool_resources.file_search)

ToolResourcesFileSearch(vector_store_ids=['vs_97QBK6E1HhTGrjrvEywUdN10'])


In [11]:
# Use the create and poll SDK helper to create a run and poll the status of
# the run until it's in a terminal state.

run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id, assistant_id=assistant.id
)

messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))

message_content = messages[0].content[0].text
annotations = message_content.annotations
citations = []
for index, annotation in enumerate(annotations):
    message_content.value = message_content.value.replace(annotation.text, f"[{index}]")
    if file_citation := getattr(annotation, "file_citation", None):
        cited_file = client.files.retrieve(file_citation.file_id)
        citations.append(f"[{index}] {cited_file.filename}")

print(message_content.value)
print("\n".join(citations))

IndexError: list index out of range